In [23]:
import numpy as np #data arrays
import pandas as pd #data manipulation and analysis
import datetime as dt #date time
import plotly.express as px #data visualisation

In [32]:
bike_share = pd.read_csv(r'C:/Users/zaydm/Documents/Repos/Google_Data_Analytics_Capstone/divvy_tripdata.csv', parse_dates=['started_at', 'ended_at'], dayfirst=True)

In [35]:
bike_share

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,E1E065E7ED285C02,classic_bike,2022-02-19 18:08:41,2022-02-19 18:23:56,State St & Randolph St,TA1305000029,Clark St & Lincoln Ave,13179,41.884621,-87.627834,41.915689,-87.634600,member
1,1602DCDC5B30FFE3,classic_bike,2022-02-20 17:41:30,2022-02-20 17:45:56,Halsted St & Wrightwood Ave,TA1309000061,Southport Ave & Wrightwood Ave,TA1307000113,41.929143,-87.649077,41.928773,-87.663913,member
2,BE7DD2AF4B55C4AF,classic_bike,2022-02-25 18:55:56,2022-02-25 19:09:34,State St & Randolph St,TA1305000029,Canal St & Adams St,13011,41.884621,-87.627834,41.879255,-87.639904,member
3,A1789BDF844412BE,classic_bike,2022-02-14 11:57:03,2022-02-14 12:04:00,Southport Ave & Waveland Ave,13235,Broadway & Sheridan Rd,13323,41.948150,-87.663940,41.952833,-87.649993,member
4,07DE78092C62F7B3,classic_bike,2022-02-16 05:36:06,2022-02-16 05:39:00,State St & Randolph St,TA1305000029,Franklin St & Lake St,TA1307000111,41.884621,-87.627834,41.885837,-87.635500,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115604,211BE0DC162D85B7,electric_bike,2022-02-23 17:47:49,2022-02-23 18:02:29,NaN,NaN,Leavitt St & Chicago Ave,18058,41.880000,-87.630000,41.895501,-87.682017,member
115605,D4D53E78000C8CA1,electric_bike,2022-02-04 10:43:47,2022-02-04 10:50:52,NaN,NaN,Leavitt St & Chicago Ave,18058,41.910000,-87.680000,41.895501,-87.682017,member
115606,9E85F07D2F94492B,electric_bike,2022-02-28 09:16:33,2022-02-28 09:28:11,Wood St & Chicago Ave,637,Canal St & Adams St,13011,41.895714,-87.672210,41.879255,-87.639904,member
115607,B61B559F81F1D823,electric_bike,2022-02-10 16:55:16,2022-02-10 16:57:53,NaN,NaN,Canal St & Adams St,13011,41.880000,-87.630000,41.879255,-87.639904,member


In [41]:
bike_share.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

In [54]:
bike_share.rename(columns= {'started_at' : 'start_time','ended_at' : 'end_time' }, inplace=True)

In [59]:
bike_share['ride_length'] = bike_share['end_time'] - bike_share['start_time']

In [60]:
bike_share


,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length
0,E1E065E7ED285C02,classic_bike,2022-02-19 18:08:41,2022-02-19 18:23:56,State St & Randolph St,TA1305000029,Clark St & Lincoln Ave,13179,41.884621,-87.627834,41.915689,-87.634600,member,0 days 00:15:15
1,1602DCDC5B30FFE3,classic_bike,2022-02-20 17:41:30,2022-02-20 17:45:56,Halsted St & Wrightwood Ave,TA1309000061,Southport Ave & Wrightwood Ave,TA1307000113,41.929143,-87.649077,41.928773,-87.663913,member,0 days 00:04:26
2,BE7DD2AF4B55C4AF,classic_bike,2022-02-25 18:55:56,2022-02-25 19:09:34,State St & Randolph St,TA1305000029,Canal St & Adams St,13011,41.884621,-87.627834,41.879255,-87.639904,member,0 days 00:13:38
3,A1789BDF844412BE,classic_bike,2022-02-14 11:57:03,2022-02-14 12:04:00,Southport Ave & Waveland Ave,13235,Broadway & Sheridan Rd,13323,41.948150,-87.663940,41.952833,-87.649993,member,0 days 00:06:57
4,07DE78092C62F7B3,classic_bike,2022-02-16 05:36:06,2022-02-16 05:39:00,State St & Randolph St,TA1305000029,Franklin St & Lake St,TA1307000111,41.884621,-87.627834,41.885837,-87.635500,member,0 days 00:02:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115604,211BE0DC162D85B7,electric_bike,2022-02-23 17:47:49,2022-02-23 18:02:29,NaN,NaN,Leavitt St & Chicago Ave,18058,41.880000,-87.630000,41.895501,-87.682017,member,0 days 00:14:40
115605,D4D53E78000C8CA1,electric_bike,2022-02-04 10:43:47,2022-02-04 10:50:52,NaN,NaN,Leavitt St & Chicago Ave,18058,41.910000,-87.680000,41.895501,-87.682017,member,0 days 00:07:05
115606,9E85F07D2F94492B,electric_bike,2022-02-28 09:16:33,2022-02-28 09:28:11,Wood St & Chicago Ave,637,Canal St & Adams St,13011,41.895714,-87.672210,41.879255,-87.639904,member,0 days 00:11:38
115607,B61B559F81F1D823,electric_bike,2022-02-10 16:55:16,2022-02-10 16:57:53,NaN,NaN,Canal St & Adams St,13011,41.880000,-87.630000,41.879255,-87.639904,member,0 days 00:02:37


In [61]:
bike_share

,ride_id,rideable_type,start_time,end_time,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length
0,E1E065E7ED285C02,classic_bike,2022-02-19 18:08:41,2022-02-19 18:23:56,State St & Randolph St,TA1305000029,Clark St & Lincoln Ave,13179,41.884621,-87.627834,41.915689,-87.634600,member,0 days 00:15:15
1,1602DCDC5B30FFE3,classic_bike,2022-02-20 17:41:30,2022-02-20 17:45:56,Halsted St & Wrightwood Ave,TA1309000061,Southport Ave & Wrightwood Ave,TA1307000113,41.929143,-87.649077,41.928773,-87.663913,member,0 days 00:04:26
2,BE7DD2AF4B55C4AF,classic_bike,2022-02-25 18:55:56,2022-02-25 19:09:34,State St & Randolph St,TA1305000029,Canal St & Adams St,13011,41.884621,-87.627834,41.879255,-87.639904,member,0 days 00:13:38
3,A1789BDF844412BE,classic_bike,2022-02-14 11:57:03,2022-02-14 12:04:00,Southport Ave & Waveland Ave,13235,Broadway & Sheridan Rd,13323,41.948150,-87.663940,41.952833,-87.649993,member,0 days 00:06:57
4,07DE78092C62F7B3,classic_bike,2022-02-16 05:36:06,2022-02-16 05:39:00,State St & Randolph St,TA1305000029,Franklin St & Lake St,TA1307000111,41.884621,-87.627834,41.885837,-87.635500,member,0 days 00:02:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115604,211BE0DC162D85B7,electric_bike,2022-02-23 17:47:49,2022-02-23 18:02:29,NaN,NaN,Leavitt St & Chicago Ave,18058,41.880000,-87.630000,41.895501,-87.682017,member,0 days 00:14:40
115605,D4D53E78000C8CA1,electric_bike,2022-02-04 10:43:47,2022-02-04 10:50:52,NaN,NaN,Leavitt St & Chicago Ave,18058,41.910000,-87.680000,41.895501,-87.682017,member,0 days 00:07:05
115606,9E85F07D2F94492B,electric_bike,2022-02-28 09:16:33,2022-02-28 09:28:11,Wood St & Chicago Ave,637,Canal St & Adams St,13011,41.895714,-87.672210,41.879255,-87.639904,member,0 days 00:11:38
115607,B61B559F81F1D823,electric_bike,2022-02-10 16:55:16,2022-02-10 16:57:53,NaN,NaN,Canal St & Adams St,13011,41.880000,-87.630000,41.879255,-87.639904,member,0 days 00:02:37
